# Processing requests

Any web application is here to provide and receive information from clients. A client sends a request to the server, and it answers with some response. Usually, the communication between these two sides is processed by the HTTP protocol using GET, POST methods and also some other types. While HTTP protocol defines multiple different methods, only GET and POST are used by web browsers. Other methods are most commonly used in REST APIs; you can read more about them in the REST API tutorial.

This topic will focus on processing GET requests in Django as they are most frequently used. As the name suggests, they are used to GET some data from the server.

- ## Processing GET requests

My friend Willy Wonka heard about Django and wanted to use it for advertising the candy his factory makes. To help him reach the goal, we will create a simple response with candies.

We assume that the name of the Django project is factory and the application name is candies.

Let's look at the project layout: 


In [ ]:
candies = {
    "Fudge":  {
        "color": "beige",
        "price": "priceless",
        "available": 100,
    },
    "Chocolate shock": {
        "color": "brown",
        "price": "precious",
        "available": 50,
    },
    "Marshmallow" : {
        "color": "pink",
        "price": "all the money in the world",
        "available": 200,
    },
}

Add this dictionary right to the top of the candies/views.py file.

On the main page, Willy wants to put only the list of what he has and nothing else. So when you go to the site, you send a GET request to his service. GET is a method to receive data from the server.

Every request to a current address of the Django application is processed with view functions. These are methods that take a request object (HttpRequest class instance) as an argument and then return a response to the client. The request object contains all the essential data about the request itself: request type, for example, and many more that we'll explore later. The response object is also an inheritor of the Django built-in class HttpResponse.

Let's add this piece of code to the candies/views.py module:



In [ ]:
from django.http import HttpResponse

def main_page_view(request):
    if request.method == "GET":
        html =  "\n".join(f"<div>{candy}</div>" for candy in candies)
        return HttpResponse(html)

View functions are just like usual Python methods. As described above, our brand new main_page_view function takes request as an argument and returns an HttpResponse object (import it from django.http).

Every view function processes a GET request by default, but to clarify the example, we're explicitly checking if the request method was GET. If the same view function is created to process multiple request methods, we can easily add more if-clauses and directions on what to do when each particular method is triggered. For example, if the request.method is POST, you may want to process some incoming data as well.

So, back to our sweet matters: passing HTML as a string to the HttpResponse class, we return a simple HTML page with a list of candies. The Django application does the rest of the work to send the data to the client. It's that simple to make a response.

## URL Routing
We create request handlers, but how does Django choose the appropriate one? In Django applications, we store all the addresses in urls.py files. Usually, there is one main urls.py located in your project directory, and each application has a separate urls.py module located as <app_name>/urls.py. Technically, you can store all your addresses inside the main urls.py file, but that's only acceptable when working on a small project. As the project grows, you'll have to store more and more URLs, and it's better to separate them instead of keeping thousands of paths in one file

- In this topic, we'll start with this concept right away. Let's create an urls.py file in your candies app and put some code there:

In [ ]:
from django.urls import path
from .views import main_page_view

urlpatterns = [
    path("candies/", main_page_view),
]

- Then add the following lines to your project's urls.py:



In [ ]:
from django.urls import include, path

urlpatterns = [
    path('', include('candies.urls')),
]

This means that all URLs defined in candies/urls.py are accessible at the root address of your Django app. So now, for example, if Willy's site has the hostname www.willywonka.com, then the assortment page will be available at the address www.willywonka.com/candies/.

To bind a link with an appropriate handler, we called the path() function and added the result to the urlpatterns list. The first argument of path() receives a string describing a link pattern that comes after the hostname. The second argument is a handler that will process a request (main_page_view in the example above)

- candies/urls.py:



In [ ]:
urlpatterns = [
    path("", main_page_view),
]

- urls.py:



In [ ]:
urlpatterns = [
    path('candies/', include('candies.urls')),
]

# Passing parameters to URLs


What if we want to display detailed information for each candy on a separate page? Of course, we could add the three new URLs to the candies/urls.py: /candies/Fudge/, /candies/Chocolate_Shock/, and /candies/Marshmallow/. All good, but what if we have a hundred different types of candies on our website? We don't want to type this many addresses by hand, do we? Especially when they look so similar.

- That's when the URL patterns come in handy. Let's add one single line to candies/urls.py (to the urlpatterns list):

In [ ]:
path("candies/<name>/", candy_info_view)

This pattern will process any request sent to /candies/any_value/. Don't forget to import candy_info_view from .views, we'll create it in a moment.

Note that Django checks all the URLs individually in the order they are placed inside the urlpatterns list until it finds a function suitable for this address. This means if at some point you'll want to create the /candies/discounts/ path, make sure to put it before the /candies/<name>/ path. Otherwise, the discounts view will never be reached.

Now, let's create a view to display detailed info on each candy. Add the following method to candies/views.py:

In [ ]:
from django.http import HttpResponse

def candy_info_view(request, name):
    title =  f"<h1>{name}</h1>"
    html =  "\n".join(f"<div>{attr}: {value}</div>" for attr, value in candies[name].items())
    return HttpResponse(title+html)

As you see, the name argument received from the URL pattern is passed as an argument to the view function.

# Class-based views

- So, we have created an app with only one type of view and only one type of request. Sometimes apps can be more complicated, with many and many different pages. In this case, it can be impractical to use separate functions for every possible request, primarily when some should be associated with others. For such cases, Django has Class-based views (CBV). It's an alternative way to implement views as Python objects instead of functions.

In [ ]:
from django.http import HttpResponse
from django.views import View

class MainPageView(View):
    def get(self, request, *args, **kwargs):
        html =  "\n".join(f"<div>{candy}</div>" for candy in candies)
        return HttpResponse(html)

class CandyView(View):
    def get(self, request, name, *args, **kwargs):
        title = f"<h1>{name}</h1>"
        html = "\n".join(f"<div>{attr}: {value}</div>" for attr, value in candies[name].items())
        return HttpResponse(title + html)

We make a new class and inherit them from view. To provide a method to handle the GET request, we define a method with the name get. This is a general rule in Django, so if you want to make a POST handler, you define a method with the name post.

Change the urls.py module, and don't forget to import our new classes:



In [ ]:
from django.urls import path
from candies.views import MainPageView, CandyView

urlpatterns = [
    path("candies/", MainPageView.as_view()),
    path("candies/<str:name>/", CandyView.as_view()),
]

For Django class-based views, we access an appropriate view function by calling the class method as_view(). This does all the work of creating an instance of the class and ensuring that the correct handler methods are called for incoming HTTP requests.

# Not Found Pages

- For each customer wishing to know more about a particular candy, make another page and class to process that request. And if the customer asks us about a nonexistent candy, we've got to report that we couldn't find it. The corresponding HTTP response status code is 404 Not Found, but we don't see any codes in the previous example.

- Django does a lot of work under the hood. The HttpResponse sets the status code 200 OK in the answer for you, which means that the communication was OK. We can either change this code in the HttpResponse class (HttpResponse(status=404)), or use the exception class Http404, which will signal to a user that they're trying to GET a page that doesn't exist.

- - Let's make a handler for a custom candy page in the same module:




In [ ]:
from django.http import HttpResponse, Http404
from django.views import View

class CandyView(View):
    def get(self, request, name, *args, **kwargs):
        if name not in candies:
            raise Http404  # or return HttpResponse(status=404)

        candy_info = "".join(
            f"<tr><td>{key}:</td><td>{value}</td></tr>"
            for key, value in candies[name].items()
        )
        return HttpResponse(f"<table><tbody>{candy_info}</tbody></table>")